In [1]:
# Query 1

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
909,application_1732639283265_0876,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "4",
        "spark.executor.memory": "2g",
        "spark.executor.cores": "4",
        "spark.driver.memory": "2g"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1882,application_1732639283265_1843,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1867,application_1732639283265_1828,pyspark,idle,Link,Link,None,
1868,application_1732639283265_1829,pyspark,idle,Link,Link,None,
1870,application_1732639283265_1831,pyspark,idle,Link,Link,None,
1871,application_1732639283265_1832,pyspark,idle,Link,Link,None,
1875,application_1732639283265_1836,pyspark,idle,Link,Link,None,
1877,application_1732639283265_1838,pyspark,idle,Link,Link,None,
1878,application_1732639283265_1839,pyspark,idle,Link,Link,None,
1880,application_1732639283265_1841,pyspark,idle,Link,Link,None,
1881,application_1732639283265_1842,pyspark,idle,Link,Link,None,
1882,application_1732639283265_1843,pyspark,idle,Link,Link,None,✔


In [27]:
# Spark DataFrame code
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, FloatType, StringType
from pyspark.sql.functions import col,desc
import time

# Initialize spark session
spark = SparkSession \
    .builder \
    .appName("DF query 1") \
    .getOrCreate()

crimes_schema = StructType([
    StructField("DR_NO", StringType(), True),
    StructField("Date Rptd", StringType(), True),
    StructField("DATE OCC", StringType(), True),
    StructField("TIME OCC", StringType(), True),
    StructField("AREA", IntegerType(), True),
    StructField("AREA NAME", StringType(), True),
    StructField("Rpt Dist No", IntegerType(), True),
    StructField("Part 1-2", IntegerType(), True),
    StructField("Crm Cd", IntegerType(), True),
    StructField("Crm Cd Desc", StringType(), True),
    StructField("Mocodes", StringType(), True),
    StructField("Vict Age", IntegerType(), True),
    StructField("Vict Sex", StringType(), True),
    StructField("Vict Descent", StringType(), True),
    StructField("Premis Cd", IntegerType(), True),
    StructField("Premis Desc", StringType(), True),
    StructField("Weapon Used Cd", IntegerType(), True),
    StructField("Weapon Desc", StringType(), True),
    StructField("Status", StringType(), True),
    StructField("Status Desc", StringType(), True),
    StructField("Crm Cd 1", IntegerType(), True),
    StructField("Crm Cd 2", IntegerType(), True),
    StructField("Crm Cd 3", IntegerType(), True),
    StructField("Crm Cd 4", IntegerType(), True),
    StructField("LOCATION", StringType(), True),
    StructField("Cross Street", StringType(), True),
    StructField("LAT", FloatType(), True),
    StructField("LON", FloatType(), True)
])

# Starting Time
start_time = time.time()

# Load the first CSV (2010-2019) into a DataFrame
df_1 = spark.read.format('csv') \
    .options(header='true') \
    .schema(crimes_schema) \
    .load("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv")

# Load the second CSV (2020-present) into a DataFrame
df_2 = spark.read.format('csv') \
    .options(header='true') \
    .schema(crimes_schema) \
    .load("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv")

# Union both DataFrames into a single DataFrame
df = df_1.union(df_2)

df.printSchema()

filtered_df = df.filter(col("Crm Cd Desc").rlike("AGGRAVATED ASSAULT"))

children_df = filtered_df.filter(col("Vict Age") < 18)
young_adults_df = filtered_df.filter((col("Vict Age") >= 18) & (col("Vict Age") <= 24))
adults_df = filtered_df.filter((col("Vict Age") >= 25) & (col("Vict Age") <= 64))
oldies_df = filtered_df.filter(col("Vict Age") > 64)

children_df.orderBy(desc("Vict Age")).select("Vict Age")
young_adults_df.orderBy(desc("Vict Age")).select("Vict Age")
adults_df.orderBy(desc("Vict Age")).select("Vict Age")
oldies_df.orderBy(desc("Vict Age")).select("Vict Age")
# Count the number of rows in the DataFrame
print(f"Number of rows in the children DataFrame: {children_df.count()}")
print(f"Number of rows in the young_adults DataFrame: {young_adults_df.count()}")
print(f"Number of rows in the adults DataFrame: {adults_df.count()}")
print(f"Number of rows in the oldies DataFrame: {oldies_df.count()}")
# Stop timing and print out the execution duration
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- DR_NO: string (nullable = true)
 |-- Date Rptd: string (nullable = true)
 |-- DATE OCC: string (nullable = true)
 |-- TIME OCC: string (nullable = true)
 |-- AREA: integer (nullable = true)
 |-- AREA NAME: string (nullable = true)
 |-- Rpt Dist No: integer (nullable = true)
 |-- Part 1-2: integer (nullable = true)
 |-- Crm Cd: integer (nullable = true)
 |-- Crm Cd Desc: string (nullable = true)
 |-- Mocodes: string (nullable = true)
 |-- Vict Age: integer (nullable = true)
 |-- Vict Sex: string (nullable = true)
 |-- Vict Descent: string (nullable = true)
 |-- Premis Cd: integer (nullable = true)
 |-- Premis Desc: string (nullable = true)
 |-- Weapon Used Cd: integer (nullable = true)
 |-- Weapon Desc: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Status Desc: string (nullable = true)
 |-- Crm Cd 1: integer (nullable = true)
 |-- Crm Cd 2: integer (nullable = true)
 |-- Crm Cd 3: integer (nullable = true)
 |-- Crm Cd 4: integer (nullable = true)
 |-- LOC

In [28]:
# Spark RDD code
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc
from pyspark import RDD
import time
import csv
from io import StringIO

# Initialize Spark session
sc = SparkSession \
    .builder \
    .appName("RDD query 1") \
    .getOrCreate() \
    .sparkContext

# Starting time
start_time = time.time()

def parse_csv(line):
    return next(csv.reader(StringIO(line)))

rdd_1 = sc.textFile("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv") \
    .map(parse_csv)


rdd_2 = sc.textFile("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv") \
    .map(parse_csv)

# Union both RDDs
rdd = rdd_1.union(rdd_2)

# Filter the initial data
filtered_rdd = rdd.filter(lambda row: "AGGRAVATED ASSAULT" in row[9])  # "Crm Cd Desc" is at index 9

# Split the filtered data into different age groups
children_rdd = filtered_rdd.filter(lambda row: int(row[11]) < 18)  # "Vict Age" is at index 11
young_adults_rdd = filtered_rdd.filter(lambda row: 18 <= int(row[11]) <= 24)
adults_rdd = filtered_rdd.filter(lambda row: 25 <= int(row[11]) <= 64)
oldies_rdd = filtered_rdd.filter(lambda row: int(row[11]) > 64)

# select the age as key and sort by key in descending orde
children_sorted = children_rdd.map(lambda x: [int(x[11]), x[0]]) \
                    .sortByKey(ascending=False)
young_adults_sorted = young_adults_rdd.map(lambda x: [int(x[11]), x[0]]) \
                    .sortByKey(ascending=False)
adults_sorted = adults_rdd.map(lambda x: [int(x[11]), x[0]]) \
                    .sortByKey(ascending=False)
oldies_sorted = oldies_rdd.map(lambda x: [int(x[11]), x[0]]) \
                    .sortByKey(ascending=False)

# Count the number of rows in the filtered RDD
print(f"Number of rows in the children RDD: {children_sorted.count()}")
print(f"Number of rows in the young_adults RDD: {young_adults_sorted.count()}")
print(f"Number of rows in the adults RDD: {adults_sorted.count()}")
print(f"Number of rows in the oldies RDD: {oldies_sorted.count()}")


# Stop timing and print out the execution duration
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of rows in the children RDD: 15928
Number of rows in the young_adults RDD: 33605
Number of rows in the adults RDD: 121093
Number of rows in the oldies RDD: 5985
Time taken: 69.66 seconds

In [29]:
# Access configuration
conf = spark.sparkContext.getConf()

# Print relevant executor settings
print("Executor Instances:", conf.get("spark.executor.instances"))
print("Executor Memory:", conf.get("spark.executor.memory"))
print("Executor Cores:", conf.get("spark.executor.cores"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Executor Instances: 4
Executor Memory: 2g
Executor Cores: 4